In [1]:
#import required functions
import warnings
warnings.filterwarnings("ignore")
import math
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import pylab as plt
import numpy as np
from IPython.display import Image
import matplotlib as matplotlib
import datetime
from pandas import DataFrame
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import xlrd
from StyleFrame import StyleFrame, Styler, utils

C:\Users\JoyThompson\AppData\Local\Continuum\anaconda3\lib\site-packages\StyleFrame\style_frame.py:11: DeprecationWarning:Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working


In [2]:
def readsheets(filenameorig, directory):
    directory1=directory+'/{}'
    xl2pd = pd.read_excel(directory1.format(filenameorig), sheetname='Month Child')
    xl2pd2 = pd.read_excel(directory1.format(filenameorig), sheetname='Month Ad')
    xl2pd3 = pd.read_excel(directory1.format(filenameorig), sheetname='Month Orders')
    xl2pdad = pd.read_excel(directory1.format(filenameorig), sheetname='Month Ad Other')

    xl2pd4 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Month Child')
    xl2pd5 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Month Ad')
    xl2pdad2 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Month Ad Other')

    xl2pd7 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Year Month Child')
    xl2pd8 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Year Month Ad')
    xl2pdad3 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Year Month Ad Other')

    xl2pd9 = pd.read_excel(directory1.format(filenameorig), sheetname='Week Child')
    xl2pd10 = pd.read_excel(directory1.format(filenameorig), sheetname='Week Ad')
    xl2pdad4 = pd.read_excel(directory1.format(filenameorig), sheetname='Week Ad Other')

    xl2pd12 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Week Child')
    xl2pd13 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Week Ad')
    xl2pdad5 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Week Ad Other')

    xl2pd15 = pd.read_excel(directory1.format(filenameorig), sheetname='Last Year Week Child')
    return xl2pd, xl2pd2, xl2pd3, xl2pdad, xl2pd4, xl2pd5, xl2pdad2, xl2pd7, xl2pd8, xl2pdad3, xl2pd9, xl2pd10, xl2pdad4, xl2pd12, xl2pd13, xl2pdad5, xl2pd15

In [3]:
def findbb(xl2pd):
    xl2pdbbsku=xl2pd.groupby(['(Parent) ASIN', '(Child) ASIN'])['Buy Box Percentage'].mean().reset_index()
    xl2pdbbsku=xl2pdbbsku.set_index('(Child) ASIN')
    xl2pdbbprod=xl2pd.groupby(['(Parent) ASIN'])['Buy Box Percentage'].mean().reset_index()
    xl2pdbbprod=xl2pdbbprod.set_index('(Parent) ASIN')
    return xl2pdbbsku, xl2pdbbprod

In [4]:
def childfix(pd):
    pdnew=pd.groupby(['(Parent) ASIN', '(Child) ASIN'])['Buy Box Percentage'].mean().reset_index()
    pd=pd.groupby(['(Parent) ASIN', '(Child) ASIN'])['Sessions', 'Units Ordered', 'Total Order Items', 'Ordered Product Sales'].sum().reset_index()
    return pd, pdnew

In [5]:
def mainkpi(xl2pd, xl2pdbbsku):
    pdsum=pd.DataFrame(columns=['Product', 'ASIN', 'Sessions', 'Items', 'Orders', 'Items per Order','Buy Box Percentage', 'Conversion', 'ASP', 
                                'AOV', 'Total Revenue'])
    skulist=[]
    for ind in xl2pd.index:
        prod=xl2pd.loc[ind, '(Parent) ASIN']
        sku=xl2pd.loc[ind, '(Child) ASIN']
        skulist.append(sku)
        sess=xl2pd.loc[ind, 'Sessions']
        items=xl2pd.loc[ind, 'Units Ordered']
        orders=xl2pd.loc[ind, 'Total Order Items']
        rev=xl2pd.loc[ind, 'Ordered Product Sales']
        bb=xl2pdbbsku.loc[sku, 'Buy Box Percentage']
        ipu=items/orders
        conv=(orders/(sess*bb))
        if conv>1:
            conv=1
        asp=rev/items
        aov=rev/orders
        pdsum=pdsum.append({'Product':prod, 'ASIN': sku, 'Sessions': sess, 'Items':items, 'Orders':orders, 
                            'Items per Order':np.round(ipu, decimals=2),'Buy Box Percentage':np.round(bb, decimals=3), 'Conversion':np.round(conv, decimals=2), 
                            'ASP':np.round(asp, decimals=2), 'AOV':np.round(aov, decimals=2), 'Total Revenue':rev},
                          ignore_index=True)
    return pdsum, skulist

In [6]:
def skuadsales(adpd, pd):
    adpdnew=adpd.groupby(['Advertised ASIN'])['7 Day Conversion Rate','Click-Thru Rate (CTR)','Cost Per Click (CPC)'].mean().reset_index()
    adpdnew=adpdnew.set_index('Advertised ASIN')
    adpd=adpd.groupby(['Advertised ASIN'])['Clicks', 'Impressions' ,'Spend'].sum().reset_index()
    adpd=adpd.set_index('Advertised ASIN')
    pd=pd.groupby(['Purchased ASIN'])['7 Day Other SKU Units (#)', '7 Day Other SKU Orders (#)',  '7 Day Other SKU Sales '].sum().reset_index()
    pd=pd.set_index('Purchased ASIN')
    
    return adpd, adpdnew, pd

In [7]:
def paidkpi(pdsum, xl2pd2, xl2pdad):
    xl2pdadlist=[]
    print(pdsum)
    for ind in xl2pdad.index:
        xl2pdadlist.append(ind)

    pdsum['Organic Sessions']=''
    pdsum['Organic Items']=''
    pdsum['Organic Orders']=''
    pdsum['Organic Items per Order']=''
    pdsum['Organic AOV']=''
    pdsum['Organic Revenue']=''
    pdsum['Paid Spend']=''
    pdsum['Paid Sessions']=''
    pdsum['Paid Items']=''
    pdsum['Paid Orders']=''
    pdsum['Paid Items per Order']=''
    pdsum['Paid AOV']=''
    pdsum['Paid Revenue']=''
    pdsum['ACoS']=''
    pdsum['TACoS']=''
    pdsum=pdsum.set_index('ASIN')
    for sku in xl2pd2.index:

        if sku in skulist:
            revtotal=pdsum.loc[sku,'Total Revenue']
            sesstotal=pdsum.loc[sku,'Sessions']
            itemtotal=pdsum.loc[sku,'Items']
            ordertotal=pdsum.loc[sku,'Orders']
            iputotal=pdsum.loc[sku,'Items per Order']
            aovtotal=pdsum.loc[sku,'AOV']

            spend=xl2pd2.loc[sku, 'Spend']
            sess=xl2pd2.loc[sku, 'Clicks']
            orgsess=sesstotal-sess 
            if orgsess < 0: 
                orgsess=0

            pdsum.loc[sku,'Paid Spend']=spend
            pdsum.loc[sku,'Paid Sessions']=sess
            pdsum.loc[sku,'Paid Spend']=spend
            pdsum.loc[sku,'Organic Sessions']=orgsess

            if sku in xl2pdadlist:
                items=xl2pdad.loc[sku,'7 Day Other SKU Units (#)']

                rev=xl2pdad.loc[sku,'7 Day Other SKU Sales ']
                orders=xl2pdad.loc[sku, '7 Day Other SKU Orders (#)']



                ipu=items/orders
                aov=rev/orders
                acos=(spend/rev)
                tacos=(spend/revtotal)
                if rev==0:
                    ipu=0
                    aov=0
                    acos=0
                    tacos=0
                orgrev=revtotal-rev    
                orgitem=itemtotal-items
                orgorder=ordertotal-orders

                if orgitem<.5:
                    orgaov=0
                    orgipu=0

                if orgitem>=.5:
                    orgipu=orgitem/orgorder
                    orgaov=orgrev/orgorder

                pdsum.loc[sku,'Organic Items']=np.round(orgitem, decimals=1)
                pdsum.loc[sku,'Organic Orders']=np.round(orgorder, decimals=1)
                pdsum.loc[sku,'Organic Items per Order']=np.round(orgipu, decimals=2)
                pdsum.loc[sku,'Organic AOV']=np.round(orgaov, decimals=2)
                pdsum.loc[sku,'Organic Revenue']=np.round(orgrev, decimals=2)
                pdsum.loc[sku, 'Paid Items']=np.round(items, decimals=1)
                pdsum.loc[sku, 'Paid Orders']=np.round(orders, decimals=1)
                pdsum.loc[sku, 'Paid Items per Order']=np.round(ipu, decimals=2)
                pdsum.loc[sku, 'Paid AOV']=np.round(aov, decimals=2)
                pdsum.loc[sku, 'Paid Revenue']=np.round(rev, decimals=2)
                pdsum.loc[sku, 'ACoS']=np.round(acos, decimals=2)
                pdsum.loc[sku, 'TACoS']=np.round(tacos, decimals=2)
    return pdsum

In [8]:
def orgkpi(pdsum):
    pdsum=pdsum.reset_index()
    for sku in pdsum.index:
        ps=pdsum.loc[sku, 'Paid Orders']
        if str(ps)=='' or int(ps)==0:
            revtotal=pdsum.loc[sku,'Total Revenue']
            sesstotal=pdsum.loc[sku,'Sessions']
            itemtotal=pdsum.loc[sku,'Items']
            ordertotal=pdsum.loc[sku,'Orders']
            iputotal=itemtotal/ordertotal
            aovtotal=revtotal/itemtotal 
            pdsum.loc[sku, 'Organic Sessions']=sesstotal
            pdsum.loc[sku,'Organic Items']=itemtotal
            pdsum.loc[sku,'Organic Orders']=ordertotal
            pdsum.loc[sku,'Organic Items per Order']=iputotal
            pdsum.loc[sku,'Organic AOV']=aovtotal
            pdsum.loc[sku,'Organic Revenue']=revtotal
            pdsum.loc[sku,'Paid Spend']=0
            pdsum.loc[sku,'Paid Sessions']=0
            pdsum.loc[sku, 'Paid Items']=0
            pdsum.loc[sku, 'Paid Orders']=0
            pdsum.loc[sku, 'Paid Items per Order']=0
            pdsum.loc[sku, 'Paid AOV']=0
            pdsum.loc[sku, 'Paid Revenue']=0
            pdsum.loc[sku, 'ACoS']=0
            pdsum.loc[sku, 'TACoS']=0
    return pdsum

In [9]:
def isprime(xl2pd3):
    xl2pd3=xl2pd3[xl2pd3['item-status']=='Shipped']
    prime=xl2pd3[xl2pd3['sales-channel']=='Amazon.com']
    noprime=xl2pd3[xl2pd3['sales-channel']=='Non-Amazon']
    primec=prime.groupby(['asin']).count().reset_index()
    primec=primec.set_index('asin')
    prime=prime.groupby(['asin'])['quantity'].sum().reset_index()
    prime=prime.set_index('asin')
    noprimec=noprime.groupby(['asin']).count().reset_index()
    noprimec=noprimec.set_index('asin')
    noprime=noprime.groupby(['asin'])['quantity'].sum().reset_index()
    noprime=noprime.set_index('asin')

    skulist2=[]
    prime['Price']=''
    prime['Orders']=''
    prime['Revenue']=''
    noprime['Price']=''
    noprime['Orders']=''
    noprime['Revenue']=''
    for ind in xl2pd3.index:
        price=xl2pd3.loc[ind, 'item-price']
        sku=xl2pd3.loc[ind, 'asin']
        if sku not in skulist2:
            skulist2.append(sku)
            if sku in prime.index:
                items=prime.loc[sku, 'quantity']
                orders=primec.loc[sku, 'purchase-date']
                prime.loc[sku, 'Price']=price
                prime.loc[sku, 'Orders' ]=orders
                prime.loc[sku, 'Revenue']=np.round(price*items, decimals=2)
            if sku in noprime.index:
                items=noprime.loc[sku, 'quantity']
                orders=noprimec.loc[sku, 'purchase-date']
                noprime.loc[sku, 'Price']=price
                noprime.loc[sku, 'Orders']=orders
                noprime.loc[sku, 'Revenue']=np.round(price*items, decimals=2)
    return prime, noprime

In [10]:
def primekpi(pdsum, prime):
    pdsum['Prime Items']=''
    pdsum['Prime Orders']=''
    pdsum['Prime Items per Order']=''
    pdsum['Prime AOV']=''
    pdsum['Prime Revenue']=''
    for ind in pdsum.index:
        sku=pdsum.loc[ind, 'ASIN']
        if sku in prime.index:
            price=prime.loc[sku, 'Price']
            orders=prime.loc[sku, 'Orders']
            items=prime.loc[sku, 'quantity']
            rev=prime.loc[sku, 'Revenue']
            pdsum.loc[ind,'Prime Items']=items
            pdsum.loc[ind,'Prime Orders']=orders
            pdsum.loc[ind,'Prime Items per Order']=np.round(items/orders, decimals=2)
            pdsum.loc[ind,'Prime AOV']=np.round(rev/orders, decimals=2)
            pdsum.loc[ind,'Prime Revenue']=rev
        else:
            pdsum.loc[ind,'Prime Items']=0
            pdsum.loc[ind,'Prime Orders']=0
            pdsum.loc[ind,'Prime Items per Order']=0
            pdsum.loc[ind,'Prime AOV']=0
            pdsum.loc[ind,'Prime Revenue']=0
    return pdsum

In [11]:
def noprimekpi(pdsum, noprime):
    pdsum['Non-Prime Items']=''
    pdsum['Non-Prime Orders']=''
    pdsum['Non-Prime Items per Order']=''
    pdsum['Non-Prime AOV']=''
    pdsum['Non-Prime Revenue']=''
    for ind in pdsum.index:
        sku=pdsum.loc[ind, 'ASIN']
        if sku in noprime.index:
            price=noprime.loc[sku, 'Price']
            orders=noprime.loc[sku, 'Orders']
            items=noprime.loc[sku, 'quantity']
            rev=noprime.loc[sku, 'Revenue']
            pdsum.loc[ind,'Non-Prime Items']=items
            pdsum.loc[ind,'Non-Prime Orders']=orders
            pdsum.loc[ind,'Non-Prime Items per Order']=np.round(items/orders, decimals=2)
            pdsum.loc[ind,'Non-Prime AOV']=np.round(rev/orders, decimals=2)
            pdsum.loc[ind,'Non-Prime Revenue']=rev
        else:
            pdsum.loc[ind,'Non-Prime Items']=0
            pdsum.loc[ind,'Non-Prime Orders']=0
            pdsum.loc[ind,'Non-Prime Items per Order']=0
            pdsum.loc[ind,'Non-Prime AOV']=0
            pdsum.loc[ind,'Non-Prime Revenue']=0
    return pdsum

In [12]:
def parentskusum(pdsum):
    pdsumparent=pdsum.groupby(['Product'])['Sessions','Items','Orders'].sum().reset_index()
    pdsumparent15=pdsum.groupby(['Product'])['Paid Sessions'].sum().reset_index()
    pdsumparent2=pdsum.groupby(['Product'])['Organic Sessions'].sum().reset_index()
    pdsumparent25=pdsum.groupby(['Product'])[ 'Organic Orders', 'Organic Items'].sum().reset_index()
    pdsumparent3=pdsum.groupby(['Product'])['Paid Spend', 'Organic Revenue', 'Paid Revenue', 'Prime Revenue'].sum().reset_index()
    pdsumparent4=pdsum.groupby(['Product'])['Paid Items', 'Paid Orders'].sum().reset_index()
    pdsumparent5=pdsum.groupby(['Product'])['Prime Items', 'Prime Orders','Non-Prime Items', 'Non-Prime Orders'].sum().reset_index()
    pdsumparent6=pdsum.groupby(['Product'])['Total Revenue','Non-Prime Revenue'].sum().reset_index()
    pdsump=pdsumparent.values
    pdsump2=pdsumparent2.values
    pdsump25=pdsumparent25.values
    pdsump15=pdsumparent15.values
    pdsump3=pdsumparent3.values
    pdsump4=pdsumparent4.values
    pdsump5=pdsumparent5.values
    pdsump6=pdsumparent6.values
    pdsumconv=pdsum.groupby(['Product'])['Conversion', 'Buy Box Percentage'].mean().reset_index()
    pdsumconv=pdsumconv.set_index('Product')

    for ind in pdsumparent.index:
        prod=pdsumparent.loc[ind, 'Product']

        bb=pdsumconv.loc[prod,'Buy Box Percentage']
        conv=pdsumconv.loc[prod, 'Conversion']
        print(bb)
        vec=pdsump[ind,1:]
        vec15=pdsump15[ind,1:]
        print(vec)
        vec2=pdsump2[ind,1:]
        vec25=pdsump25[ind,1:]
        vec3=pdsump3[ind,1:]
        print(vec3)
        vec4=pdsump4[ind,1:]
        vec5=pdsump5[ind,1:]
        vec6=pdsump6[ind,1:]
        if vec[2]!=0:
            ipo=np.round(vec[1]/vec[2], decimals=2)
            asp=np.round(vec6[0]/vec[1], decimals=2)
            aov=np.round(vec6[0]/vec[2], decimals=2)
        else:
            ipo=0
            asp=0
            aov=0
        orgunits=vec25[1]
        orgorders=vec25[0]

        if vec25[0]!=0:
            ipoorg=np.round(vec25[1]/vec25[0], decimals=2)
            aovorg=np.round(vec3[1]/vec25[0], decimals=2)
        else:
            ipoorg=0
            aovorg=0
        if vec3[2]!=0:
            ipop=np.round(vec4[0]/vec4[1], decimals=2)
            aovp=np.round(vec3[2]/vec4[1], decimals=2)
            
            acos=np.round(vec3[0]/vec3[2], decimals=2)
            tacos=np.round(vec3[0]/vec6[0], decimals=2)
        else:
            ipop=0
            aovp=0
            acos=0
            tacos=0

        if vec5[1]!=0:
            ipoprime=np.round(vec5[0]/vec5[1], decimals=2)
            aovprime=np.round(vec3[3]/vec5[1], decimals=2)
        else:
            ipoprime=0
            aovprime=0
        if vec5[3]!=0:
            iponp=np.round(vec5[2]/vec5[3], decimals=2)
            aovnp=np.round(vec6[1]/vec5[3], decimals=2)
        else:
            iponp=0
            aovnp=0


        pdsum=pdsum.append({'Product':prod, 'ASIN':'Parent Item Total', 'Sessions':vec[0], 'Items':vec[1], 'Orders':vec[2], 'Items per Order':ipo, 'Buy Box Percentage':bb,'Conversion':conv,
                            'ASP':asp, 'AOV':aov, 'Total Revenue':vec6[0],'Organic Sessions':vec2[0], 
                                            'Organic Items':orgunits, 'Organic Orders':orgorders, 'Organic Items per Order':ipoorg, 'Organic AOV':aovorg, 'Organic Revenue':vec3[1], 'Paid Spend':vec3[0],
                                          'Paid Sessions':vec15[0], 'Paid Items':vec4[0], 'Paid Orders':vec4[1], 'Paid Items per Order':ipop, 'Paid AOV':aovp,
                                          'Paid Revenue':vec3[2], 'ACoS':acos, 'TACoS':tacos, 'Prime Items':vec5[0], 'Prime Orders':vec5[1], 'Prime Items per Order':ipoprime,
                                          'Prime AOV':aovprime, 'Prime Revenue':vec3[3], 'Non-Prime Items':vec5[2], 'Non-Prime Orders':vec5[3], 'Non-Prime Items per Order':iponp,
                                          'Non-Prime AOV':aovnp, 'Non-Prime Revenue':vec6[1]}, ignore_index=True)
    pdsum=pdsum.sort_values(['Product','ASIN'])
    pdsum=pdsum.set_index('Product')
    pdsum=pdsum.reset_index()
    return pdsum

In [13]:
def fixmom(pd2,pd, pdad):
    pd2=pd2.reset_index()
    pd2sum=pd2.set_index('Advertised ASIN').sum()
    pdsum=pd.set_index('(Child) ASIN').sum()
    pdad=pdad.reset_index()
    pdadsum=pdad.set_index('Purchased ASIN').sum()
    return pd2sum, pdsum, pdadsum

In [14]:
def createpdmom():
    pdmom=pd.DataFrame(columns=['Source', 'KPI/Data','This Month, Last Year', 'Previous Month', 'Current Month',
                        'YoY Difference', 'MoM Difference'])
    pdmom['KPI/Data']=['Ad Spend $', 'Ad Clicks', 'Ad Impressions', 'Ad Transactions', 'Ad Revenue $', 'Ad AOV $', 'Ad Items Per Order', 'CPC $', 'CTR %',
              'ACoS %','Sessions/Visitors', 'Transactions', 'Revenue $', 'AOV $', 'Items Per Order', 'TACoS %']
    pdmom.loc[0,'Source']='Paid Advertising'
    pdmom.loc[10,'Source']='Total Account'
    pdmom=pdmom.reset_index()
    pdmom=pdmom.drop(columns=['index'],axis=0)
    return pdmom

In [15]:
def momyoywow(pd, pd2,pd3, col,pdmomwow):
    kpilist=[pd[2], pd[0],pd[1],pd3[1],pd3[2],float(pd3[2]/pd3[1]), float(pd3[0]/pd3[1]), float(pd[2]/pd[0]),float(pd[0]/pd[1]),
             float(pd[2]/pd3[2]),pd2[1],pd2[3],pd2[4],float(pd2[4]/pd2[3]),float(pd2[2]/pd2[3]),float(pd[2]/pd2[4])]
    pdmomwow[col]=kpilist
    return pdmomwow

In [16]:
def calcmomyoy(pdmom):
    for ind in pdmom.index:
        thismonth=pdmom.loc[ind, 'Current Month']
        thismonth=np.round(float(thismonth),decimals=3)
        pdmom.loc[ind, 'Current Month']=thismonth
        lastmonth=pdmom.loc[ind,'Previous Month']
        lastmonth=np.round(float(lastmonth),decimals=3)
        pdmom.loc[ind, 'Previous Month']=lastmonth
        lastymonth=pdmom.loc[ind, 'This Month, Last Year']
        lastymonth=np.round(float(lastymonth),decimals=3)
        pdmom.loc[ind, 'This Month, Last Year']=lastymonth
        yoy=(thismonth-lastymonth)/lastymonth
        mom=(thismonth-lastmonth)/lastmonth 
        pdmom.loc[ind, 'YoY Difference']=np.round(float(yoy), decimals=2)
        pdmom.loc[ind, 'MoM Difference']=np.round(float(mom), decimals=2)
    pdmom=pdmom.set_index('Source')
    return pdmom

In [17]:
def createpdwow():
    pdwow=pd.DataFrame(columns=['Source','KPI/Data','This Week, Last Year','Last Week, This Year','This Week, This Year','YoY Difference', 'WoW Difference'])
    pdwow['KPI/Data']=['Ad Spend $', 'Ad Clicks', 'Ad Impressions', 'Ad Transactions', 'Ad Revenue $', 'Ad AOV $', 'Ad Items Per Order', 'CPC $', 'CTR %',
              'ACoS %','Sessions/Visitors', 'Transactions', 'Revenue $', 'AOV $', 'Items Per Order', 'TACoS %']
    weeklist=['This Month, Last Year','Last Week, This Year','This Week, This Year']
    pdwow.loc[0,'Source']='Paid Advertising'
    pdwow.loc[10,'Source']='Total Account'
    pdwow=pdwow.reset_index()
    pdwow=pdwow.drop(columns=['index'],axis=0)
    return pdwow

In [18]:
def calcwowyoy(pdwow):
    for ind in pdwow.index:
        thisweek=pdwow.loc[ind, 'This Week, This Year']
        thisweek=np.round(float(thisweek),decimals=3)
        pdwow.loc[ind, 'This Week, This Year']=thisweek
        lastweek=pdwow.loc[ind, 'Last Week, This Year']
        lastweek=np.round(float(lastweek),decimals=3)
        pdwow.loc[ind, 'Last Week, This Year']=lastweek
        lastyweek=pdwow.loc[ind, 'This Week, Last Year']
        lastyweek=np.round(float(lastyweek),decimals=3)
        pdwow.loc[ind, 'This Week, Last Year']=lastyweek
        yoy=(thisweek-lastyweek)/lastyweek
        wow=(thisweek-lastweek)/lastweek
        pdwow.loc[ind, 'YoY Difference']=np.round(float(yoy), decimals=2)
        pdwow.loc[ind, 'WoW Difference']=np.round(float(wow), decimals=2)
    pdwow=pdwow.set_index('Source')
    return pdwow
    

In [19]:
def colorpd(pdsku):
    general=['Sessions', 'Items', 'Orders', 'Items per Order','Buy Box Percentage', 'Conversion', 'ASP', 'AOV', 'Total Revenue']
    org=['Organic Sessions', 'Organic Items', 'Organic Orders', 'Organic Items per Order', 'Organic AOV', 'Organic Revenue']
    paid=['Paid Spend', 'Paid Sessions', 'Paid Items', 'Paid Orders','Paid Items per Order', 'Paid AOV', 'Paid Revenue', 
         'ACoS', 'TACoS']
    prime=['Prime Sessions', 'Prime Items', 'Prime Orders', 'Prime Items per Order','Prime AOV','Prime Revenue']
    nprime=['Non-Prime Sessions', 'Non-Prime Items', 'Non-Prime Orders', 'Non-Prime Items per Order','Non-Prime AOV','Non-Prime Revenue']

    def highlight_cols(pd):
        pdout=pd.copy()
        pdout[pdout['ASIN']=='Parent Item Total'] = 'background-color: #CCD1D1'

        pdout[general] = 'background-color: #D6EAF8'
        pdout[org] = 'background-color: #FCF3CF'
        pdout[paid] = 'background-color: #D1F2EB'
        pdout[prime] = 'background-color: #FDEBD0'

        pdout[nprime] = 'background-color: #E8DAEF'
        return pdout
    pdout=pdsku.style.format({"Total Revenue": "${:20,.0f}","ASP": "${:20,.0f}", "AOV": "${:20,.0f}",
                        "Organic Revenue": "${:20,.0f}","Organic AOV": "${:20,.0f}",
                        "Paid Spend": "${:20,.0f}", "Paid Revenue": "${:20,.0f}", "Paid AOV": "${:20,.0f}",
                        "Prime AOV": "${:20,.0f}", "Prime Revenue": "${:20,.0f}",
                        "Non-Prime AOV": "${:20,.0f}", "Non-Prime Revenue": "${:20,.0f}"}).apply(highlight_cols, axis=None).set_caption('SKU Level Analysis').hide_index()
    return pdout

In [20]:
def savepd(pdsum, pdmom, pdwow, client):
    filenamefinal= '/Users/JoyThompson/Desktop/ClientReporting/CompletedReports/'+ client + ' Weekly & Monthly Reporting.xlsx'                                     
    writer = pd.ExcelWriter(filenamefinal)
    pdsum.to_excel(writer, sheet_name='Weekly & Monthly Reporting')
    pdmom.to_excel(writer, sheet_name='Monthly - Account Overview')
    pdwow.to_excel(writer, sheet_name='Weekly - Account Overview')
    writer.save()
    return filenamefinal
  

In [21]:
directory='/Users/JoyThompson/Desktop/ClientReporting/ClientData'
for filenameorig in os.listdir(directory):
    client = filenameorig.split(' ')
    client=client[0]
    xl2pd, xl2pd2, xl2pd3, xl2pdad, xl2pd4, xl2pd5, xl2pdad2, xl2pd7, xl2pd8, xl2pdad3, xl2pd9, xl2pd10, xl2pdad4, xl2pd12, xl2pd13, xl2pdad5, xl2pd15=readsheets(filenameorig, directory)
    
    xl2pdbbsku, xl2pdbbprod = findbb(xl2pd)
    xl2pd, xl2pdbb=childfix(xl2pd)
    xl2pd4, xl2pdbb4=childfix(xl2pd4)
    xl2pd7, xl2pdbb7=childfix(xl2pd7)
    xl2pd9, xl2pdbb9=childfix(xl2pd9)
    xl2pd12, xl2pdbb12=childfix(xl2pd12)
    xl2pd15, xl2pdbb15=childfix(xl2pd15)
    pdsum, skulist= mainkpi(xl2pd, xl2pdbbsku)
    xl2pd2, xl2pdm, xl2pdad=skuadsales(xl2pd2, xl2pdad)
    xl2pd5, xl2pd5m,xl2pdad2=skuadsales(xl2pd5, xl2pdad2)
    xl2pd8, xl2pd8m,xl2pdad3=skuadsales(xl2pd8, xl2pdad3)
    xl2pd10, xl2pd10m,xl2pdad4=skuadsales(xl2pd10, xl2pdad4)
    xl2pd13, xl2pd13m,xl2pdad5=skuadsales(xl2pd13, xl2pdad5)
    pdsum=paidkpi(pdsum, xl2pd2, xl2pdad)
    pdsum=orgkpi(pdsum)
    prime, noprime = isprime(xl2pd3)
    pdsum = primekpi(pdsum, prime)
    pdsum = noprimekpi(pdsum, noprime)
    pdsum=parentskusum(pdsum)
    xl2pd2sum, xl2pdsum, xl2pdadsum=fixmom(xl2pd2, xl2pd, xl2pdad)
    xl2pd5sum, xl2pd4sum,xl2pdadsum2=fixmom(xl2pd5, xl2pd4, xl2pdad2)
    xl2pd8sum, xl2pd7sum, xl2pdadsum3=fixmom(xl2pd8, xl2pd7, xl2pdad3)
    xl2pd10sum, xl2pd9sum,xl2pdadsum4=fixmom(xl2pd10, xl2pd9, xl2pdad4)
    xl2pd13sum, xl2pd12sum,xl2pdadsum5=fixmom(xl2pd13, xl2pd12, xl2pdad5)
    xl2pd15sum=xl2pd15.set_index('(Child) ASIN').sum()
    pdmom=createpdmom()
    pdmom=momyoywow(xl2pd2sum,xl2pdsum,xl2pdadsum,'Current Month',pdmom)
    pdmom=momyoywow(xl2pd5sum,xl2pd4sum,xl2pdadsum2,'Previous Month',pdmom)
    pdmom=momyoywow(xl2pd8sum,xl2pd7sum,xl2pdadsum3,'This Month, Last Year',pdmom)
    pdmom=calcmomyoy(pdmom)
    pdwow=createpdwow()
    pdwow=momyoywow(xl2pd10sum,xl2pd9sum,xl2pdadsum4,'This Week, This Year',pdwow)
    pdwow=momyoywow(xl2pd13sum,xl2pd12sum,xl2pdadsum5,'Last Week, This Year',pdwow)
    pdwow['This Week, Last Year']=[0,0,0,0,0,0,0,0,0,0,xl2pd15sum[1],xl2pd15sum[3],xl2pd15sum[4],
                float(xl2pd15sum[4]/xl2pd15sum[3]),float(xl2pd15sum[2]/xl2pd15sum[3]),0]
    pdwow=calcwowyoy(pdwow)
    
    filenamefinal=savepd(pdsum, pdmom, pdwow, client)

C:\Users\JoyThompson\AppData\Local\Continuum\anaconda3\lib\site-packages\botocore\awsrequest.py:624: DeprecationWarning:Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working


       Product        ASIN  Sessions   Items  Orders  Items per Order  \
0   B007WFSGMU  B007WFSGMU   11289.0  1663.0  1267.0             1.31   
1   B00C79SR70  B00C79SR70    1024.0    58.0    52.0             1.12   
2   B00FE4BODO  B00FE4BODO      14.0     1.0     1.0             1.00   
3   B00FE6GLLW  B00FE6GLLW     418.0   139.0   136.0             1.02   
4   B00G05DYTI  B00G05DYTI    2193.0   555.0   377.0             1.47   
5   B00GOIIAJ0  B00GOIIAJ0     243.0    13.0    13.0             1.00   
6   B00U9PFLOG  B00U9PFLOG      56.0     3.0     3.0             1.00   
7   B00UY4WUNC  B00UY4WUNC      86.0    13.0    11.0             1.18   
8   B00UY4Z9K8  B00UY4Z9K8     161.0    17.0    17.0             1.00   
9   B00UY50ZJ2  B00UY50ZJ2      56.0     5.0     5.0             1.00   
10  B00UY54RRI  B00UY54RRI     162.0    30.0    18.0             1.67   
11  B00UY56UM8  B00UY56UM8     605.0    27.0    27.0             1.00   
12  B00UY58S7S  B00UY58S7S     120.0    15.0    14.

In [25]:
filenamefinal= '/Users/JoyThompson/Desktop/ClientReporting/CompletedReports/'+ 'DammitDolls' + ' Weekly & Monthly Reporting.xlsx'            
pdsum=pd.read_excel(filenamefinal, sheet_name='Weekly & Monthly Reporting')
pdmom=pd.read_excel(filenamefinal, sheet_name='Monthly - Account Overview')
pdwow=pd.read_excel(filenamefinal, sheet_name='Weekly - Account Overview')

general=['Sessions', 'Items', 'Orders', 'Items per Order','Buy Box Percentage', 'Conversion', 'ASP', 'AOV', 'Total Revenue']
org=['Organic Sessions', 'Organic Items', 'Organic Orders', 'Organic Items per Order', 'Organic AOV', 'Organic Revenue']
paid=['Paid Spend', 'Paid Sessions', 'Paid Items', 'Paid Orders','Paid Items per Order', 'Paid AOV', 'Paid Revenue', 
         'ACoS', 'TACoS']
prime=['Prime Sessions', 'Prime Items', 'Prime Orders', 'Prime Items per Order','Prime AOV','Prime Revenue']
nprime=['Non-Prime Sessions', 'Non-Prime Items', 'Non-Prime Orders', 'Non-Prime Items per Order','Non-Prime AOV','Non-Prime Revenue']  
    +
filenamefinal= '/Users/JoyThompson/Desktop/ClientReporting/CompletedReports/'+ client + ' Weekly & Monthly Reporting.xlsx'
writer = StyleFrame.ExcelWriter(filenamefinal)

sfsum=StyleFrame(pdsum)
sfmom=StyleFrame(pdmom)
sfwow=StyleFrame(pdwow)

sfsum.apply_column_style(cols_to_style=['Product', 'ASIN'], styler_obj=Styler(bg_color='#FDFEFE', bold=True, font=utils.fonts.arial,font_size=8))

sfsum.apply_headers_style(styler_obj=Styler(bg_color=utils.colors.black, bold=True, font_size=10,
                                                font_color=utils.colors.white))
sfsum.to_excel(writer, sheet_name='Sheet1')
writer.save()
    